In [204]:
import pandas as pd

In [205]:
# read in the dataset
# change the filepath to the location of yelp_academic_dataset_business.json on your local machine
# this file is too big to upload to github, so if you need to download it you can do so here: https://www.yelp.com/dataset
yelp_precovid = pd.read_json("data/yelp_academic_dataset_business.json", lines=True)

In [206]:
# original yelp dataset has 209,393 records
yelp_precovid.shape

(209393, 14)

In [207]:
# subset the data to only the eight states with a sufficient number of businesses
states = ["AZ", "IL", "NC", "NV", "OH", "PA", "SC", "WI"]
yelp_precovid = yelp_precovid[yelp_precovid["state"].isin(states)]

In [208]:
# after subsetting to the eight states: 153,760 records
# removed 55,633 records
yelp_precovid.shape

(153760, 14)

In [209]:
# remove rows with missing postal code
yelp_precovid = yelp_precovid[yelp_precovid["postal_code"] != ""]

In [210]:
# generate a dataframe of zip codes in the yelp data
yelp_zips = yelp_precovid.rename({"business_id": "precovid_count"}, axis = "columns").groupby(["postal_code"]).count()["precovid_count"]
yelp_zips = pd.DataFrame(yelp_zips)
# subset to only zip codes with at least 50 yelp businesses
yelp_zips = yelp_zips[yelp_zips["precovid_count"] > 50]

In [211]:
# subset the yelp dataset to only the zip codes with at least 50 yelp businesses
yelp_precovid = yelp_precovid[yelp_precovid["postal_code"].isin(yelp_zips.index.values)]
# write out precovid yelp csv for reference
yelp_precovid.to_csv("data/yelp_precovid.csv")

In [212]:
# after subsetting to the zip codes with at least 50 businesses: 149,766 records
# removed 3994 records
yelp_precovid.shape

(149766, 14)

In [213]:
# add the number of open businesses to the yelp_zips data
yelp_zips = yelp_zips.join(pd.DataFrame(yelp_precovid.groupby(["postal_code"])["is_open"].sum())).rename({"is_open": "precovid_open"}, axis = "columns")

In [214]:
# write out yelp zip csv for reference
yelp_zips.to_csv("data/yelp_zips.csv")